In [1]:
import time
from openai import OpenAI
client = OpenAI()

In [2]:
# file upload
batch_input_file = client.files.create(
  file=open("../data/ch02_batchinput.jsonl", "rb"),
  purpose="batch"
)

# Create a new batch
client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "테스트를 위한 배치",
    }
)
start_time = time.time()

In [13]:
# Batch 객체들의 리스트를 가져옵니다.
batches = client.batches.list().data

# 각 배치의 상세 정보를 출력합니다.
for batch in batches[:5]:
    print(f"Batch ID: {batch.id}")
    print(f"Status: {batch.status}")
    print(f"Created At: {batch.created_at}")
    print(f"input_file_id: {batch.input_file_id}")
    print(f"output_file_id: {batch.output_file_id}")
    print("------")

Batch ID: batch_67445ef4f3e4819094e37d2b8eef6d44
Status: completed
Created At: 1732534004
input_file_id: file-DfwxQAsxRrDieLTvMdU8QS
output_file_id: file-NVMy8b4cCGjwSVJW6ST47f
------
Batch ID: batch_67443bc16cf08190a7246b9cb492ce01
Status: in_progress
Created At: 1732524993
input_file_id: file-JVgBMZfhVEAzUnFXwHsAnw
output_file_id: None
------
Batch ID: batch_674338f7552881908768f70dae8e5ea9
Status: completed
Created At: 1732458743
input_file_id: file-Q8mYsoemnm5DxZzjiPHUZY
output_file_id: file-PVsQEdu7xe2Qe1uHSzEUXQ
------
Batch ID: batch_674336d52b688190b0d397478747745f
Status: completed
Created At: 1732458197
input_file_id: file-Cj8Lt9CWeyuJZ925C2ktKF
output_file_id: file-7ztiejaK3fWFWvfVBxZqUg
------
Batch ID: batch_674335e4d4ac81909430e1e4f05dc8a3
Status: completed
Created At: 1732457956
input_file_id: file-L88QzYT6zQbzhDtsH8GCJa
output_file_id: file-T2RKu1GkBaRZvsbmKPZMDM
------


In [5]:
# status를 15초에 한번씩 확인하고 'completed'일때까지 로그를 출력
# status가 'completed'가 되면 루프를 탈출합니다.
# 총 걸린 시간을 추가로 출력합니다.
while True:
  status = client.batches.retrieve(client.batches.list().data[0].id).status
  print("time: ", time.strftime('%X', time.localtime()), "   |    status: ", status)
  if status == "completed":
    break
  time.sleep(60) # 60초에 한번씩 확인

print("Total time: ", time.time() - start_time)

# output file을 다운로드 받아서 출력
file_response = client.files.content(client.batches.list().data[0].output_file_id)
print(file_response.text.encode('utf-8').decode('unicode-escape'))

time:  20:27:43    |    status:  completed
Total time:  58.700679779052734
{"id": "batch_req_67445f08ee8c8190a67820139f242258", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "5b54e823cfed0fe0a4cf5b443c6a3c45", "body": {"id": "chatcmpl-AXRWoeiADhlkTOy0CZjg6LKGP9G86", "object": "chat.completion", "created": 1732534006, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "[첫번쨰 성공!]", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 47, "completion_tokens": 8, "total_tokens": 55, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "system_fingerprint": "fp_0705bf87c0"}}, "error": null}
{"id": "batch_req_67445f09016c8190b596ee73a0e8ba8e", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "1fae76e32f

In [6]:
file_response.text.encode('utf-8').decode('unicode-escape').split("\n")[0].split(",")[10]

' "content": "[첫번쨰 성공!]"'

In [12]:
# 응답 내용을 파일로 저장
with open("../data/ch02_batchoutput.jsonl", "w", encoding="utf-8") as f:
    f.write(file_response.text.encode('utf-8').decode('unicode-escape'))